<a href="https://colab.research.google.com/github/onism/MyLearning/blob/master/NLP_basic_tf2_0_sms_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os 
import io

tf.__version__

'2.2.0'

In [2]:
path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip",
                  origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",
                  extract=True)

!unzip $path_to_zip -d data

204800/203415 [==============================] - 1s 3us/step
Archive:  /root/.keras/datasets/smsspamcollection.zip
  inflating: data/SMSSpamCollection  
  inflating: data/readme             


In [3]:
lines = io.open('data/SMSSpamCollection').read().strip().split('\n')
lines[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [4]:
# pre-process data

spam_dataset = []
count = 0
for line in lines:
    label, text = line.split('\t')
    if label.lower().strip() == 'spam':
        spam_dataset.append((1, text.strip()))
    else:
        spam_dataset.append((0, text.strip()))

print(spam_dataset[0])

(0, 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')


In [5]:
#data normalization

import pandas as pd 
df = pd.DataFrame(spam_dataset, columns=['Spam', 'Message'])
df.head()

,Spam,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
import re 

def message_length(x):
    return len(x)

def num_captitals(x):
    _, count = re.subn(r'[A-Z]','',x)
    return count 

def num_punctuation(x):
    _, count = re.subn(r'\W', '', x)
    return count 

df['Capitals'] = df['Message'].apply(num_captitals)
df['Punctuation'] = df['Message'].apply(num_punctuation)
df['Length'] = df['Message'].apply(message_length)

df.describe()

,Spam,Capitals,Punctuation,Length
count,5574.000000,5574.000000,5574.000000,5574.000000
mean,0.134015,5.621636,18.942591,80.443488
std,0.340699,11.683233,14.825994,59.841746
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


In [7]:
train = df.sample(frac=0.8, random_state=42)
test = df.drop(train.index)

In [8]:
train.describe()

,Spam,Capitals,Punctuation,Length
count,4459.000000,4459.000000,4459.000000,4459.000000
mean,0.132765,5.519399,18.886522,80.316439
std,0.339359,11.405424,14.602023,59.346407
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,35.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


In [9]:
test.describe()

,Spam,Capitals,Punctuation,Length
count,1115.000000,1115.000000,1115.000000,1115.000000
mean,0.139013,6.030493,19.166816,80.951570
std,0.346116,12.731059,15.694599,61.807655
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,28.000000,123.000000
max,1.000000,127.000000,195.000000,790.000000


In [10]:
# model building 

def make_model(input_dims=3, num_units=12):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(num_units, input_dim=input_dims, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

x_train = train[['Length', 'Punctuation', 'Capitals']]
y_train = train[['Spam']]

x_test = test[['Length', 'Punctuation', 'Capitals']]
y_test = test[['Spam']]

x_train

,Length,Punctuation,Capitals
3690,25,4,1
3527,161,48,107
724,40,7,1
3370,69,17,3
468,37,8,1
...,...,...,...
3280,444,114,44
3186,65,14,50
3953,81,23,2
2768,38,8,2


In [11]:
type(x_train)

pandas.core.frame.DataFrame

In [12]:
model = make_model()

model.fit(x_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
279/279 [==============================] - 0s 964us/step - loss: 6.7936 - accuracy: 0.5580
Epoch 2/10
279/279 [==============================] - 0s 1ms/step - loss: 0.4431 - accuracy: 0.8652
Epoch 3/10
279/279 [==============================] - 0s 1ms/step - loss: 0.4115 - accuracy: 0.8675
Epoch 4/10
279/279 [==============================] - 0s 1ms/step - loss: 0.3806 - accuracy: 0.8856
Epoch 5/10
279/279 [==============================] - 0s 986us/step - loss: 0.3531 - accuracy: 0.8881
Epoch 6/10
279/279 [==============================] - 0s 982us/step - loss: 0.3421 - accuracy: 0.8894
Epoch 7/10
279/279 [==============================] - 0s 1ms/step - loss: 0.3163 - accuracy: 0.8959
Epoch 8/10
279/279 [==============================] - 0s 979us/step - loss: 0.3061 - accuracy: 0.8948
Epoch 9/10
279/279 [==============================] - 0s 979us/step - loss: 0.2930 - accuracy: 0.8986
Epoch 10/10
279/279 [==============================] - 0s 988us/step - loss: 0.2896 - accu

In [13]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 956us/step - loss: 0.2729 - accuracy: 0.8951


[0.2728753983974457, 0.8950672745704651]

In [14]:
y_train_pred = model.predict(x_train)

tf.math.confusion_matrix(tf.constant(y_train.Spam), y_train_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3867,    0],
       [ 592,    0]], dtype=int32)>

In [15]:
# tokenization and stop word remove

!pip install stanfordnlp

     |████████████████████████████████| 163kB 2.5MB/s 


In [16]:
import stanfordnlp as snlp

en = snlp.download('en')

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
Y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [00:23<00:00, 10.2MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


In [17]:
en = snlp.Pipeline(lang='en')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand

In [18]:
sentence = 'Go until jurong point, crazy.. Available only in bugis n great world'
tokenized = en(sentence)

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [21]:
tokenized.sentences[0].tokens

[<Token index=1;words=[<Word index=1;text=Go;lemma=go;upos=VERB;xpos=VB;feats=Mood=Imp|VerbForm=Fin;governor=0;dependency_relation=root>]>,
 <Token index=2;words=[<Word index=2;text=until;lemma=until;upos=ADP;xpos=IN;feats=_;governor=4;dependency_relation=case>]>,
 <Token index=3;words=[<Word index=3;text=jurong;lemma=jurong;upos=PROPN;xpos=NNP;feats=Number=Sing;governor=4;dependency_relation=compound>]>,
 <Token index=4;words=[<Word index=4;text=point;lemma=point;upos=NOUN;xpos=NN;feats=Number=Sing;governor=1;dependency_relation=obl>]>,
 <Token index=5;words=[<Word index=5;text=,;lemma=,;upos=PUNCT;xpos=,;feats=_;governor=1;dependency_relation=punct>]>,
 <Token index=6;words=[<Word index=6;text=crazy;lemma=crazy;upos=ADJ;xpos=JJ;feats=Degree=Pos;governor=1;dependency_relation=parataxis>]>,
 <Token index=7;words=[<Word index=7;text=..;lemma=..;upos=PUNCT;xpos=.;feats=_;governor=1;dependency_relation=punct>]>]

In [22]:
for snt in tokenized.sentences:
    for word in snt.tokens:
        print(word.text)
    print('<End of Sentence>')

Go
until
jurong
point
,
crazy
..
<End of Sentence>
Available
only
in
bugis
n
great
world
<End of Sentence>


In [ ]:
def word_counts(x, pipeline=en):
    doc = pipeline(x)
    count = sum(  [len(sentence.tokens) for sentence in doc.sentences]  )
    return count 

df['Words'] = df['Message'].apply(word_counts)

df.describe()

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

In [ ]:
train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

In [ ]:
x_train = train[ ['Length', 'Punctuation', 'Capitals', 'Words']  ]
x_test = test[ ['Length', 'Punctuation', 'Capitals', 'Words'] ]
y_train = train['Spam']
y_test = test['Spam']

model = make_model(input_dims=4)

model.fit(x_train, y_train, epochs=10, batch_size=10)